In [ ]:
import matplotlib.pyplot as plt
import ROOT 
import numpy as np
import CanvasSetup as cSetup
import os,copy
import itertools as itrTools

In [ ]:
import ROOT 
import operator 
import array

from CanvasSetup import *

def getFitParams(hist0):
    mean  = hist0.GetMean()
    stdDev= hist0.GetStdDev()
    minX=mean-10*stdDev
    maxX=mean+10*stdDev
    hist0.Fit("gaus","Q0","",minX,maxX);
    f=hist0.GetFunction("gaus");
    if f:
        return [f.GetParameter(1),f.GetParameter(2),f]
    else:
        return [-1.0,1.0,None]
    
def saveTheDictionary(aCollection,fname=None,folder=None):
    outfile=folder
    if fname!=None and outfile==None:
        outfile=ROOT.TFile(fname,"RECREATE")
    outfile.cd()
    for key in aCollection:
#         print("Doing : ",key)
        if type(aCollection[key])==type({}):
            print("\tMakdig dir : ",key)
            inFolder=outfile.mkdir(key)
            saveTheDictionary(aCollection[key],None,inFolder)
        else:
            aCollection[key].Write()
    if fname!=None:
        outfile.Close()

colurWheel=[ 
	   ROOT.kBlack  ,
	   ROOT.kRed    ,
	   ROOT.kGreen  ,
       ROOT.kBlue   ,
	   ROOT.kMagenta ,
	   ROOT.kYellow ,
	   ROOT.kOrange ,
       ROOT.kSpring ,
	   ROOT.kCyan   ,
	   ROOT.kAzure   ,
	   ROOT.kTeal   ,
	   ROOT.kGray    ,
	   ROOT.kViolet ,
	   ROOT.kPink   ,
    ]

class Plot:
    def __init__(self, **args):
        self.name        = args.get("Name", "plot")
        self.legend      = args.get("Legend",None)
        self.histo       = args.get("Histo", None)
        self.fit         = args.get("Fit", None)
        self.markerColor = args.get("MarkerColor", ROOT.kBlue+2)
        self.markerStyle = args.get("MarkerStyle", 22)
        self.markerSize = args.get("MarkerSize", 5)
        self.lineColor   = args.get("LineColor", ROOT.kBlue+2)
        self.lineStyle   = args.get("LineStyle", 1)
        self.lineWidth   = args.get("LineWidth", 1)
        self.options   = args.get("Options", "")
        self.drawLegend   = args.get("DrawLegend",False)
        self.drawLine     = args.get("DrawLine", False)
        self.histo.SetName(self.name+"_histo")
        self.normalize =args.get("Normalize",False)
        self.scaleFactor =args.get("ScaleFactor",None)
        self.doFit =args.get("doFit",False)
        self.drawFit =args.get("doFit",False)
        self.verbose =args.get("verbose",True)

class PlotCanvas:
    def __init__(self, **args):
        self.name  = ""
        self.plots = []
        self.rates = []
        self.plotDir =  args.get("prefix","plots/")
        self.yRange = (0.0, 1.0)
        self.xRange = (0, 100)
        self.yRange2 = (0.0, 1.0)
        self.xTitle = args.get("xTitle","E_{T}^{e offl} [GeV]")
        self.yTitle = args.get("yTitle","a.u")
        self.yTitle2 = args.get("yTitle","a.u")
        self.legendPosition = (0.4,0.2,0.9,0.6)
        self.descPosition = (0.6,0.58)
        self.desc = args.get("desc",["L1 Efficiency"])
        self.logx = args.get("logx",False)
        self.logy = args.get("logy",False)
        self.logy2 = args.get("logy2",False)
        self.canvasParams = args.get("canvasPars",getDefaultCanvas())    
        self.setPlotStyle()

    def addRate(self, rate):
        self.rates.append(rate)
    def addPlot(self, plot):
        self.plots.append(plot)
    def clearPlots(self):
        self.plots=[]


    def plot(self):
        xmax=0.0
        xmin=1e9
        xmax_=1e9
        xmin_=0.0
        ymax=0.0
        ymin=1e9
        ymax_=1e9
        ymin_=0.0
        for plot in self.plots:
            histo = plot.histo
            print("Scale Factor  : before  = ",plot.scaleFactor)    
            if histo.Integral() ==0 :
                continue
            if plot.normalize:
                x= histo.Integral()
                if x==0:
                    x=1.0
                if plot.scaleFactor==None:
                    plot.scaleFactor=1.0/( x)
                else:
                    plot.scaleFactor/=(x)
                print("Integral = = ",1.0/(x))    
            if plot.scaleFactor!=None:
                histo.Scale(plot.scaleFactor)
            print("Scaled the histogram by a factor of : ",plot.scaleFactor)
            print("             Integral after Scaling : ",histo.Integral())
            
            if self.yRange[1]=='auto':
                ymax_=histo.GetMaximum()
                if ymax < ymax_*1.1:
                    ymax=ymax_*1.1
            if self.yRange[0]=='auto': 
                ymin_=histo.GetMinimum()
                if ymin > ymin_*0.8:
                    ymin=ymin_*0.8
            
            if self.xRange[1]=='auto':
                xmax_=histo.GetXaxis().GetBinCenter( histo.GetNbinsX() ) + 0.5*histo.GetBinWidth(histo.GetNbinsX())
                print(xmax_,xmax)
                if abs(xmax) < abs(xmax_):
                    xmax=xmax_
            if self.xRange[0]=='auto': 
                xmin_=histo.GetXaxis().GetBinCenter(1) - 0.5*histo.GetBinWidth(1)
                print(xmin_,xmin)
                if abs(xmin) > abs(xmin_):
                    xmin=xmin_
        
        if self.yRange[0]=='auto':
            self.yRange=(ymin,self.yRange[1])
        if self.yRange[1]=='auto':
            self.yRange=(self.yRange[0],ymax)
        if self.xRange[0]=='auto':
            self.xRange=(xmin,self.xRange[1])
        if self.xRange[1]=='auto':
            self.xRange=(self.xRange[0],xmax)

#         print("Setting  X-RANGE  : ",self.xRange)
#         print("Setting  Y-RANGE  : ",self.yRange)
        
#         print("xmax xmin : ",xmin,xmax)
        canvas = ROOT.TCanvas("c_"+self.name, self.name, 700, 800)
        canvas.SetGrid()
        hDummy = ROOT.TH1F("hDummy_"+self.name, self.name, 1, self.xRange[0],self.xRange[1])

        #print("ymin_,ymax_",ymin_,ymax_,"ymin , ymax - > " ,ymin,ymax)
            #print( "\tY : Min - > Max  : " ,histo.GetMinimum()," -> ", histo.GetMaximum())
        hDummy.SetAxisRange(self.yRange[0], self.yRange[1], "Y")
        hDummy.SetXTitle(self.xTitle)
        hDummy.SetYTitle(self.yTitle)
        hDummy.Draw("C")
        if self.logx : canvas.SetLogx()
        if self.logy : canvas.SetLogy()
        self.logz= True
        if self.logz : canvas.SetLogz()
        
        
        CMSbox=self.getCMSBox()
        extraTextBox=self.getExtraTextBox()
        lumibox=self.getLumiBox()


        selbox=[]
        n=0
        for inx in range(len(self.desc)):
            selbox.append(ROOT.TLatex  (self.descPosition[0], self.descPosition[1] -n*0.04, self.desc[n]))
            selbox[-1].SetNDC()
            selbox[-1].SetTextSize(0.035)
            selbox[-1].SetTextFont(12)
            selbox[-1].SetTextAlign(13)
            n+=1
        for plot in self.plots:
            if(plot.doFit):
                histo = plot.histo
                fitP  = getFitParams(histo)
                strFit='('+'{0:0.3f}'.format(fitP[0])+ ' , '+'{0:0.3f}'.format(fitP[1]) +')'
                selbox.append(ROOT.TLatex  (self.descPosition[0], self.descPosition[1] -n*0.04, strFit))
                selbox[-1].SetNDC()
                selbox[-1].SetTextSize(0.035)
                selbox[-1].SetTextFont(12)
                selbox[-1].SetTextColor(plot.lineColor)
                selbox[-1].SetTextAlign(13)
                if plot.drawFit and  fitP[2]:
                    fitP[2].SetLineStyle(plot.lineStyle)
                    fitP[2].SetLineWidth(2)
                    fitP[2].SetLineColor(plot.lineColor)
                    fitP[2].Draw("same")
                n+=1

        #Line legend
        legend = ROOT.TLegend(self.legendPosition[0],self.legendPosition[1],self.legendPosition[2],self.legendPosition[3])
        legend.SetTextFont(42)
        legend.SetFillColor(0)

        for plot in self.plots:
            histo = plot.histo
            print("Ploting hist : ",histo.GetName())
            if plot.verbose:
                print( "\tX : Min - > Max  : " ,histo.GetBinCenter(1)," -> ", histo.GetBinCenter(histo.GetNbinsX())," [ ",histo.GetBinCenter(2)-histo.GetBinCenter(1)," ]" )
                print( "\tY : Min - > Max  : " ,histo.GetMinimum()," -> ", histo.GetMaximum())
                
            histo.SetMarkerStyle(plot.markerStyle)
            histo.SetMarkerSize(plot.markerSize)
            histo.SetMarkerColor(plot.markerColor)
            histo.SetLineStyle(plot.lineStyle)
            histo.SetLineWidth(plot.lineWidth)
            histo.SetLineColor(plot.lineColor)
            #histo.Draw("same e")
#             print(plot.options)
            histo.Draw("same "+plot.options)
#             print( histo.GetOption())

            if(plot.legend):
                legend.AddEntry(histo, plot.legend, "pe")
                legend.SetTextSize(0.025) 
            if(plot.drawLegend) :  legend.Draw()
        
        scale = (self.yRange[1] -self.yRange[0])/(self.yRange2[1] - self.yRange2[0])
        y20=self.yRange2[0]

        if(self.logy2):
            scale = (self.yRange[1] -self.yRange[0])/(ROOT.log(self.yRange2[1]) - ROOT.log(self.yRange2[0]) )
            y20 = ROOT.log(self.yRange2[0])


        #####   begin :            For plotting a graph in the Second Axis         #####

        if len(self.rates) >0 :
            axis=None
            canvas.SetRightMargin(0.18);
            if(self.logy2):
                axis = ROOT.TGaxis(self.xRange[1],self.yRange[0],self.xRange[1], self.yRange[1],self.yRange2[0],self.yRange2[1],510,"+LG")
            else :
                axis = ROOT.TGaxis(self.xRange[1],self.yRange[0],self.xRange[1], self.yRange[1],self.yRange2[0],self.yRange2[1],510,"+L")
            axis.SetLineColor(ROOT.kRed);
            axis.SetTextColor(ROOT.kRed);
            axis.SetTitle(self.yTitle2)
            axis.SetTitleColor(ROOT.kBlack)
            axis.SetTitleOffset(1.2)
            axis.Draw();

        for rate in self.rates:
            histo = rate.histo
            if self.logy2:
                for i in range(histo.GetNbinsX()):
                    content=histo.GetBinContent(i+1)
                    scaledContent = ( ROOT.log(content + 1e-20 )  - y20 )* scale
                    histo.SetBinContent(i+1,scaledContent)
                    #print("\t ",content," -> ",scaledContent,"  = ( ",ROOT.log(content + 1e-20),"  - ",y20," )* ",scale)
            else:
                for i in range(histo.GetNbinsX()):
                    content=histo.GetBinContent(i+1)
                    scaledContent = (content  - y20 )* scale
                    histo.SetBinContent(i+1,scaledContent)
            
            histo.SetMarkerStyle(rate.markerStyle)
            histo.SetMarkerColor(rate.markerColor)
            histo.SetLineColor(rate.markerColor)
            histo.Draw("pl same "+ rate.options)
            legend.AddEntry(histo, rate.legend, "pe")
            legend.SetTextSize(0.025) 
            if(rate.drawLegend) :  legend.Draw()
        
        #####   end :            For plotting a graph in the Second Axis         #####
            
        CMSbox.Draw()
        extraTextBox.Draw()
        lumibox.Draw()
        for selb in selbox:
            selb.Draw()
        
#         canvas.Print(self.plotDir+"/"+self.name+".pdf", "pdf")
        canvas.Print(self.plotDir+"/"+self.name+".png", "png")

        return canvas

    def getCMSBox(self):
        par=self.canvasParams["CMS"]
        CMSbox       = ROOT.TLatex  (par["xpos"],par["ypos"], par["text"])
        CMSbox.SetNDC()
        CMSbox.SetTextSize(par["textSize"])
        CMSbox.SetTextFont(par["textFont"])
        CMSbox.SetTextColor(ROOT.kBlack)
        CMSbox.SetTextAlign(13) 
        return CMSbox
   
    def getExtraTextBox(self):
        par=self.canvasParams["TAG"]
        extraTextBox = ROOT.TLatex  (par["xpos"], par["ypos"], par["text"])
        extraTextBox.SetNDC()
        extraTextBox.SetTextSize(par["textSize"])
        extraTextBox.SetTextFont(par["textFont"])
        extraTextBox.SetTextColor(ROOT.kBlack)
        extraTextBox.SetTextAlign(13)
        
        return extraTextBox

    def getLumiBox(self):
        par=self.canvasParams["EnLumi"]
        lumibox = ROOT.TLatex  ( par["xpos"], par["ypos"], par["text"])
        lumibox.SetNDC()
        lumibox.SetTextAlign(31)
        lumibox.SetTextSize(par["textSize"])
        lumibox.SetTextFont(par["textFont"])
        lumibox.SetTextColor(ROOT.kBlack)
        
        return lumibox
        
    def setPlotStyle(self):
        ROOT.gROOT.SetStyle("Plain")
        ROOT.gStyle.SetOptStat()
        ROOT.gStyle.SetOptFit(0)
        ROOT.gStyle.SetOptTitle(0)
        ROOT.gStyle.SetFrameLineWidth(2)
        ROOT.gStyle.SetPadBottomMargin(0.13)
        ROOT.gStyle.SetPadLeftMargin(0.15)
        ROOT.gStyle.SetPadTopMargin(0.06)
        ROOT.gStyle.SetPadRightMargin(0.05)

        ROOT.gStyle.SetLabelFont(42,"X")
        ROOT.gStyle.SetLabelFont(42,"Y")
        ROOT.gStyle.SetLabelSize(0.04,"X")
        ROOT.gStyle.SetLabelSize(0.04,"Y")
        ROOT.gStyle.SetLabelOffset(0.01,"Y")
        ROOT.gStyle.SetTickLength(0.02,"X")
        ROOT.gStyle.SetTickLength(0.02,"Y")
        ROOT.gStyle.SetLineWidth(2)
        ROOT.gStyle.SetTickLength(0.02 ,"Z")

        ROOT.gStyle.SetTitleSize(0.1)
        ROOT.gStyle.SetTitleFont(42,"X")
        ROOT.gStyle.SetTitleFont(42,"Y")
        ROOT.gStyle.SetTitleSize(0.05,"X")
        ROOT.gStyle.SetTitleSize(0.05,"Y")
        ROOT.gStyle.SetTitleOffset(1.1,"X")
        ROOT.gStyle.SetTitleOffset(1.4,"Y")
        ROOT.gStyle.SetOptStat(0)
        ROOT.gStyle.SetPalette(1)
        ROOT.gStyle.SetPaintTextFormat("3.2f")
        ROOT.gROOT.ForceStyle()

col=[ 0,ROOT.kBlack,ROOT.kRed,ROOT.kBlue,ROOT.kGreen,6,7,8,9,10,11,12,13,14,15  ]

def getDefaultPlotParams(col=1,marker=22,markerSize=1):
    return {'Legend':"Data : 2018, Unpacked",
                          'MarkerColor' : col,
                          'LineColor':col,
                          'LineWidth':2,
                          'LineStyle':9,
                          'MarkerStyle':marker,
                          'MarkerSize':markerSize,
                          'DrawLegend':True,
                          'Option':'pc'
            } 

def getDefaultPlot(prefix='plots/',name='defaultName',cPars=cSetup.getCanvasParams('GEN')):
#     plot=putil.PlotCanvas(prefix=prefix,canvasPars=cPars)
    plot=PlotCanvas(prefix=prefix,canvasPars=cPars)
    
    plot.name   = name
    plot.xRange = (3.0,50.0)  
    plot.yRange = (0.0,1.02) 
    plot.yTitle = "#epsilon"  
    plot.xTitle = "E_{T}^{offl.}"
    plot.desc   = ["L1 EG Isolation"] 
    plot.legendPosition = (0.6,0.5,0.90,0.65)
    plot.descPosition   = (0.6,0.75)     
    plot.logx = False
    plot.logy = False
    return plot

def getTheObjectsFromFile(aFile):
    histStore={}
    for key in aFile.GetListOfKeys():
        kk=key.GetName()
        curObj=aFile.Get(kk)
        if curObj.Class_Name()=='TDirectoryFile':
            histStore[kk]=getTheObjectsFromFile(curObj)
        else:
            histStore[kk]=curObj       
    return histStore

def getCanvasParams(tag,lumi="-1 fb"):
    cPars=getDefaultCanvas()
    print("Canvas Tag : ", tag )
    if tag=="2018Full":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "SingleEG 2018 (58.9fb^{-1}) 13 TeV"
        return cPars
    if tag=="Run3MC":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "Run 3 MC, 13,6 TeV"
        return cPars
    if tag=="Run3DataX":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "Run 3 Egamma"
    if tag=="Run3Data":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "Run 3 Egamma ( "+lumi+")"
        return cPars
     
    return cPars


def getCanvasParams(tag,lumi="-1 fb"):
    cPars=getDefaultCanvas()
    print("Canvas Tag : ", tag )
    if tag=="2018Full":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "SingleEG 2018 (58.9fb^{-1}) 13 TeV"
        return cPars
    if tag=="Run3MC":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "Run 3 MC, 13,6 TeV"
        return cPars
    if tag=="Run3Data":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "Run 3 Egamma ( "+lumi+")"
        return cPars
     
    return cPars

In [ ]:
metadata=None
with open('data/turnonFnames.json', 'r') as fp:
    metadata = json.load(fp)
    
fnamesTurnOns=metadata['fnames']   
legend=metadata['legend']    



In [ ]:
filesToLoad=[
    'dataR3Unpaked_postCalib',
    'data2018',
    
    'run3Data2022EWithHybridV3OldIsoVsSCEt',
    'run3Data2022EWithHybridV3OldIsoVsPFEt',
    'run3Data2022EWithHybridV3NewIsoVsSCEt',
    'run3Data2022EWithHybridV3NewIsoVsPFEt',
    
    'run3Data2022EDefaultOldIsoVsSCEt',
    'run3Data2022EDefaultOldIsoVsPFEt',
    'run3Data2022EDefaultNewIsoVsSCEt',
    'run3Data2022EDefaultNewIsoVsPFEt',
 ]

In [ ]:
histStoreTurnOns={}
histFileStoreTurnOns={}
for tag in filesToLoad:
    if tag in histFileStoreTurnOns:
        histFileStore[tag].Close()
    histFileStoreTurnOns[tag]=ROOT.TFile(fnamesTurnOns[tag],'READ')
    print(tag)
    if tag in ['data2018']:
        histStoreTurnOns[tag]=getTheObjectsFromFile(histFileStoreTurnOns[tag])
    else:
        histStoreTurnOns[tag]=getTheObjectsFromFile(histFileStoreTurnOns[tag])['0']

In [ ]:
for i in histStoreTurnOns['run3MCWithGenEtVsGenEt']:
#     print(i)
    if 'Efficiency' not in i:
        continue
    if 'Graph_' not in i:
        continue
    print(i)


In [ ]:
tagsToPlot=[]
if False: # Data : vsPFEt [ hybrid vs Default  | new Iso ]
    tagsToPlot=[
        'run3Data2022EWithHybridV3NewIsoVsPFEt',
        'run3Data2022EDefaultNewIsoVsPFEt',
    ]   
    colours={
        'run3Data2022EWithHybridV3NewIsoVsPFEt' : ROOT.kRed,
        'run3Data2022EDefaultNewIsoVsPFEt' : ROOT.kBlue,
    }
    legend={
        "run3Data2022EWithHybridV3NewIsoVsPFEt": "Hybrid Calib.,Data",
        "run3Data2022EDefaultNewIsoVsPFEt"   : "Default Calib.,Data",
    }
    
    baseCanvas='Run3DataX'
    varToChose='Offline'
    # varToChose='GEN'
    # varToChose='SC'

    prefix='results/plots/v7/data2022E/hybridV3/turnons_newIso_HybVsDef/'
    prefix+=varToChose+'/'
    lumi='4.35 /fb'
    if not os.path.exists(prefix):
        os.system('mkdir -p '+prefix) 
        
if False:  # Data : vsPFEt [ hybrid | new vs old Iso ]
    tagsToPlot=[
        'run3Data2022EWithHybridV3OldIsoVsPFEt',
        'run3Data2022EWithHybridV3NewIsoVsPFEt',
    ]   
    colours={
        'run3Data2022EWithHybridV3OldIsoVsPFEt' : ROOT.kRed,
        'run3Data2022EWithHybridV3NewIsoVsPFEt' : ROOT.kBlue,
    }
    legend={
        "run3Data2022EWithHybridV3OldIsoVsPFEt": "Deafault Iso.",
        "run3Data2022EWithHybridV3NewIsoVsPFEt": "New Iso.",
    }
    
    baseCanvas='Run3DataX'
    varToChose='Offline'
    # varToChose='GEN'
    # varToChose='SC'

    prefix='results/plots/v7/data2022E/hybridV3/turnons_Hybrid_DefVsNew_iso/'
    prefix+=varToChose+'/'
    lumi='4.35 /fb'
    if not os.path.exists(prefix):
        os.system('mkdir -p '+prefix)      
        
        
if True:  # Data : vsSC [ hybrid | new vs old Iso ]
    tagsToPlot=[
        'run3Data2022EWithHybridV3OldIsoVsSCEt',
        'run3Data2022EWithHybridV3NewIsoVsSCEt',
    ]   
    colours={
        'run3Data2022EWithHybridV3OldIsoVsSCEt' : ROOT.kRed,
        'run3Data2022EWithHybridV3NewIsoVsSCEt' : ROOT.kBlue,
    }
    legend={
        "run3Data2022EWithHybridV3OldIsoVsSCEt": "Deafault Iso.",
        "run3Data2022EWithHybridV3NewIsoVsSCEt": "New Iso.",
    }
    
    baseCanvas='Run3DataX'
    varToChose='Offline'
    # varToChose='GEN'
    varToChose='SC'

    prefix='results/plots/v7/data2022E/hybridV3/turnons_Hybrid_DefVsNew_iso/'
    prefix+=varToChose+'/'
    lumi='4.35 /fb'
    if not os.path.exists(prefix):
        os.system('mkdir -p '+prefix)              

In [ ]:
# histNew={}
# for tag in ['run3DataWithHybridVsPFEt','run3DataWithHybridVsSCEt','run3DataWithDefaultNewIsoVsSCEt']:
#     histNew[tag]={}
#     for i in histStoreTurnOns[tag]:
#         ni=i
#         if 'oose' in i:
#             ni=i.replace('Loose','Tight').replace('loose','tight')
#         elif 'ight' in i:
#             ni=i.replace('Tight','Loose').replace('tight','loose')
#         histNew[tag][ni]=histStoreTurnOns[tag][i]
#     histStoreTurnOns[tag]=histNew[tag]
    

## Pt Efficiency

# Gen Et scale as a function of GenEt

In [ ]:
if True:# Vs PU
    plots=[]
    cPars=getCanvasParams(baseCanvas,lumi)
    rBin=1
   
    plot=getDefaultPlot(prefix=prefix,name='effVsPU',cPars=cPars) 
    plot.legendPosition = (0.5,0.41,0.85,0.52)
    plot.descPosition   = (0.55,0.75)
    plot.desc =["L1 EG","Loose Offline Electron","|#eta| <2.4","E_{T}^{"+varToChose+"} > 40 , E_{T}^{L1} > 32"]
    plot.yTitle = "#epsilon"  
    plot.xTitle = "PU" 
    plot.xRange = (5.0,100.0) 
    plot.logx = False
    plot.yRange = (0.0,1.01) 
    plots.append(plot) 

    i=2
    hname='Graph_from_hist_L1_32vsPUoffline40GeVEfficiency'
    
    for tag in tagsToPlot: #['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
        print(tag)
        hist=histStoreTurnOns[tag][hname].Clone()
        pparams=getDefaultPlotParams(col=i,marker=22)
        pparams['Legend']=legend[tag]
        pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag]
        pparams['Options']='pe'
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor =None
        aplot.doFit=False ; aplot.drawFit=True
        aplot.verbose= False
        aplot.drawLegend = True
        plots[-1].addPlot(aplot)
    
    canvas = []
    for plot in plots:
        canvas.append(plot.plot()) 
    

In [ ]:
if True:# Barrel vs Edcap Inclusive
    plots=[]
    cPars=getCanvasParams(baseCanvas,lumi)
    rBin=1
    for eT in [32,34,36]:
        plot=getDefaultPlot(prefix=prefix,name='effComponentBarrel_'+str(eT)+varToChose,cPars=cPars)  
        plot.legendPosition = (0.60,0.35,0.85,0.60)
        plot.descPosition   = (0.65,0.75)
        plot.desc =["E_{T} > "+str(eT),"Barrel"]
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{"+varToChose+"}"
        plot.xRange = (5.0,500.0) 
        plot.logx = True
        plot.yRange = (0.0,1.01) 
        plots.append(plot) 

        i=2
        hname='Graph_from_hist_L1Et'+str(eT)+'_default_BarrelEfficiency'
    
        for tag in tagsToPlot:#['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
            hist=histStoreTurnOns[tag][hname].Clone()
    #         hist.Rebin(rBin)
            pparams=getDefaultPlotParams(col=i,marker=22)
            pparams['Legend']=legend[tag]
            pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag]
            pparams['Options']='pe'
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = False ;aplot.scaleFactor =None
            aplot.doFit=False ; aplot.drawFit=True
            aplot.verbose= False
            plots[-1].addPlot(aplot)
        
        
        plot=getDefaultPlot(prefix=prefix,name='effComponentECap_'+str(eT)+varToChose,cPars=cPars) 
        plot.legendPosition = (0.60,0.35,0.85,0.60)
        plot.descPosition   = (0.65,0.75)
        plot.desc =["E_T > "+str(eT),"End Cap"]
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{L1} / E_{T}^{"+varToChose+"}"
        plot.xRange = (5.0,500.0) 
        plot.logx = True
        plot.yRange = (0.0,1.01) 
        plots.append(plot) 
        hname='Graph_from_hist_L1Et'+str(eT)+'_default_ECapEfficiency'
    
        for tag in tagsToPlot:#['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
            hist=histStoreTurnOns[tag][hname].Clone()
    #         hist.Rebin(rBin)
            pparams=getDefaultPlotParams(col=i,marker=22)
            pparams['Legend']=legend[tag]
            pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag]
            pparams['Options']='pe'
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = False ;aplot.scaleFactor =None
            aplot.doFit=False ; aplot.drawFit=True
            aplot.verbose= False
            plots[-1].addPlot(aplot)
            
    
        plot=getDefaultPlot(prefix=prefix,name='effComponentInclusive_'+str(eT)+varToChose,cPars=cPars) 
        plot.legendPosition = (0.60,0.35,0.85,0.60)
        plot.descPosition   = (0.65,0.75)
        plot.desc =["E_{T} > "+str(eT),"Inclusive"]
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{L1} / E_{T}^{"+varToChose+"}"
        plot.xRange = (5.0,500.0) 
        plot.logx = True
        plot.yRange = (0.0,1.01) 
        plots.append(plot) 
        hname='Graph_from_hist_L1Et'+str(eT)+'_defaultEfficiency'
    
        for tag in tagsToPlot:#['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
            hist=histStoreTurnOns[tag][hname].Clone()
    #         hist.Rebin(rBin)
            pparams=getDefaultPlotParams(col=i,marker=22)
            pparams['Legend']=legend[tag]
            pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag]
            pparams['Options']='pe'
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = False ;aplot.scaleFactor =None
            aplot.doFit=False ; aplot.drawFit=True
            aplot.verbose= False
            plots[-1].addPlot(aplot)


    canvas = []
    for plot in plots:
        canvas.append(plot.plot()) 
    

In [ ]:
if True:# Barrel vs Edcap Tight
    plots=[]
    cPars=getCanvasParams('Run3Data',lumi)
    rBin=1
    for eT in [28,30]:
        plot=getDefaultPlot(prefix=prefix,name='effComponentBarrel_tightIso_'+str(eT)+varToChose,cPars=cPars) 
        plot.legendPosition = (0.60,0.48,0.85,0.60)
        plot.descPosition   = (0.65,0.75)
        plot.desc =["E_T > "+str(eT),"Tight Isolation","Barrel"]
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{"+varToChose+"}"
        plot.xRange = (5.0,80.0) 
        plot.logx = False
        plot.yRange = (0.0,1.01) 
        plots.append(plot) 

        i=2
        hname='Graph_from_hist_L1Et'+str(eT)+'_tightiso_BarrelEfficiency'
    
        for tag in tagsToPlot:#['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
            hist=histStoreTurnOns[tag][hname].Clone()
    #         hist.Rebin(rBin)
            pparams=getDefaultPlotParams(col=i,marker=22)
            pparams['Legend']=legend[tag]
            pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag]
            pparams['Options']='pe'
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = False ;aplot.scaleFactor =None
            aplot.doFit=False ; aplot.drawFit=True
            aplot.verbose= False
            plots[-1].addPlot(aplot)
        
        
        plot=getDefaultPlot(prefix=prefix,name='effComponentECap_tight_'+str(eT)+varToChose,cPars=cPars) 
        plot.legendPosition = (0.60,0.35,0.85,0.60)
        plot.descPosition   = (0.65,0.75)
        plot.desc =["E_T > "+str(eT),"Tight Isolation","Endcap"]
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{"+varToChose+"}"
        plot.xRange = (5.0,100.0) 
        plot.logx = False
        plot.yRange = (0.0,1.01) 
        plots.append(plot) 
        hname='Graph_from_hist_L1Et'+str(eT)+'_tightiso_ECapEfficiency'
    
        for tag in tagsToPlot:#['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
            hist=histStoreTurnOns[tag][hname].Clone()
    #         hist.Rebin(rBin)
            pparams=getDefaultPlotParams(col=i,marker=22)
            pparams['Legend']=legend[tag]
            pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag]
            pparams['Options']='pe'
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = False ;aplot.scaleFactor =None
            aplot.doFit=False ; aplot.drawFit=True
            aplot.verbose= False
            plots[-1].addPlot(aplot)

    canvas = []
    for plot in plots:
        canvas.append(plot.plot()) 
    

In [ ]:
if True:# Barrel vs Edcap Loose
    plots=[]
    cPars=getCanvasParams('Run3Data',lumi)
    rBin=1
    for eT in [15,22,25]:
        plot=getDefaultPlot(prefix=prefix,name='effComponentBarrel_looseIso_'+str(eT)+varToChose,cPars=cPars) 
        plot.legendPosition = (0.60,0.45,0.85,0.60)
        plot.descPosition   = (0.65,0.75)
        plot.desc =["E_T_DoubleEG_"+str(eT-10)+"_"+str(eT),"Loose Isolation","Barrel"]
        plot.desc =["E_{T} > "+str(eT),"Loose Isolation","Barrel"]
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{"+varToChose+"}"
        plot.xRange = (5.0,80.0) 
        plot.logx = False
        plot.yRange = (0.0,1.01) 
        plots.append(plot) 

        i=2
        hname='Graph_from_hist_L1Et'+str(eT)+'_looseiso_BarrelEfficiency'
    
        for tag in tagsToPlot:#['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
            hist=histStoreTurnOns[tag][hname].Clone()
    #         hist.Rebin(rBin)
            pparams=getDefaultPlotParams(col=i,marker=22)
            pparams['Legend']=legend[tag]
            pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag]
            pparams['Options']='pe'
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = False ;aplot.scaleFactor =None
            aplot.doFit=False ; aplot.drawFit=True
            aplot.verbose= False
            plots[-1].addPlot(aplot)
        
        
        plot=getDefaultPlot(prefix=prefix,name='effComponentECap_loose_'+str(eT)+varToChose,cPars=cPars) 
        plot.legendPosition = (0.60,0.45,0.85,0.60)
        plot.descPosition   = (0.65,0.75)
        plot.desc =["E_T > "+str(eT),"Loose Isolation","Endcap"]
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{L1} / E_{T}^{"+varToChose+"}"
        plot.xRange = (5.0,100.0) 
        plot.logx = False
        plot.yRange = (0.0,1.01) 
        plots.append(plot) 
        hname='Graph_from_hist_L1Et'+str(eT)+'_looseiso_ECapEfficiency'
    
        for tag in tagsToPlot:#['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
            hist=histStoreTurnOns[tag][hname].Clone()
    #         hist.Rebin(rBin)
            pparams=getDefaultPlotParams(col=i,marker=22)
            pparams['Legend']=legend[tag]
            pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag]
            pparams['Options']='pe'
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = False ;aplot.scaleFactor =None
            aplot.doFit=False ; aplot.drawFit=True
            aplot.verbose= False
            plots[-1].addPlot(aplot)


    canvas = []
    for plot in plots:
        canvas.append(plot.plot()) 
    

In [ ]:
col=[ROOT.kGreen+2,ROOT.kBlue, ROOT.kRed]
if True:  # Inclusive Loose vs Tight
    plots=[]
    cPars=getCanvasParams('Run3Data',lumi)
    rBin=1
    for tag in  tagsToPlot : #['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
        for eT in [15,28,30,36]:
            plot=getDefaultPlot(prefix=prefix,name='tightVsLoose_'+str(eT)+'_'+tag+'_'+varToChose,cPars=cPars) 
            plot.legendPosition = (0.65,0.51,0.90,0.60)
            plot.descPosition   = (0.65,0.75)
            plot.desc =["E_T > "+str(eT),legend[tag]]
            plot.yTitle = "#epsilon"  
            plot.xTitle = "E_{T}^{L1} / E_{T}^{"+varToChose+"}"
            plot.xRange = (5.0,500.0) 
            plot.logx = True
            plot.yRange = (0.0,1.01) 
            plots.append(plot) 

            i=0
            hname='Graph_from_hist_L1Et'+str(eT)+'_defaultEfficiency'
            hist=histStoreTurnOns[tag][hname].Clone()
    #         hist.Rebin(rBin)
            pparams=getDefaultPlotParams(col=i,marker=22)
            pparams['Legend']='Inclusive'
            pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i]
            pparams['Options']='pe'
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = False ;aplot.scaleFactor =None
            aplot.doFit=False ; aplot.drawFit=True
            aplot.verbose= False
            plots[-1].addPlot(aplot)


            i=1
            hname='Graph_from_hist_L1Et'+str(eT)+'_looseiso_defEfficiency'
            hist=histStoreTurnOns[tag][hname].Clone()
    #         hist.Rebin(rBin)
            pparams=getDefaultPlotParams(col=i,marker=22)
            pparams['Legend']='Loose Isolation'
            pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i]
            pparams['Options']='pe'
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = False ;aplot.scaleFactor =None
            aplot.doFit=False ; aplot.drawFit=True
            aplot.verbose= False
            plots[-1].addPlot(aplot)

            i=2
            hname='Graph_from_hist_L1Et'+str(eT)+'_tightiso_defEfficiency'
            hist=histStoreTurnOns[tag][hname].Clone()
    #         hist.Rebin(rBin)
            pparams=getDefaultPlotParams(col=i,marker=22)
            pparams['Legend']='Tight Isolation'
            pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i]
            pparams['Options']='pe'
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = False ;aplot.scaleFactor =None
            aplot.doFit=False ; aplot.drawFit=True
            aplot.verbose= False
            plots[-1].addPlot(aplot)


        canvas = []
        for plot in plots:
            canvas.append(plot.plot())


In [ ]:
col=[ROOT.kGreen+2,ROOT.kBlue, ROOT.kRed]
tagsSC=['run3DataWithHybridVsSCEt','run3DataDefaultVsSCEt']
if True:  # Inclusive Loose vs Tight 20-40
    plots=[]
    cPars=getCanvasParams('Run3Data',lumi)
    rBin=1
    for tag in tagsSC+['data2018']:#['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
        for eT in [28,30,34]:
            plot=getDefaultPlot(prefix=prefix,name='tightVsLoose20TO40_'+str(eT)+'_'+tag+'_'+varToChose,cPars=cPars) 
            plot.legendPosition = (0.17,0.51,0.40,0.60)
            plot.descPosition   = (0.17,0.75)
            plot.desc =["E_T > "+str(eT),legend[tag]]
            plot.yTitle = "#epsilon"  
            plot.xTitle = "E_{T}^{L1} / E_{T}^{"+varToChose+"}"
            plot.xRange = (20.0,40.0) 
            plot.logx = False
            plot.yRange = (0.0,1.01) 
            plots.append(plot) 

            i=0
            hname='Graph_from_hist_L1Et'+str(eT)+'_defaultEfficiency'
            hist=histStoreTurnOns[tag][hname].Clone()
    #         hist.Rebin(rBin)
            pparams=getDefaultPlotParams(col=i,marker=22)
            pparams['Legend']='Inclusive'
            pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i]
            pparams['Options']='pe'
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = False ;aplot.scaleFactor =None
            aplot.doFit=False ; aplot.drawFit=True
            aplot.verbose= False
            plots[-1].addPlot(aplot)


            i=1
            hname='Graph_from_hist_L1Et'+str(eT)+'_looseiso_defEfficiency'
            hist=histStoreTurnOns[tag][hname].Clone()
    #         hist.Rebin(rBin)
            pparams=getDefaultPlotParams(col=i,marker=22)
            pparams['Legend']='Loose Isolation'
            pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i]
            pparams['Options']='pe'
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = False ;aplot.scaleFactor =None
            aplot.doFit=False ; aplot.drawFit=True
            aplot.verbose= False
            plots[-1].addPlot(aplot)

            i=2
            hname='Graph_from_hist_L1Et'+str(eT)+'_tightiso_defEfficiency'
            hist=histStoreTurnOns[tag][hname].Clone()
    #         hist.Rebin(rBin)
            pparams=getDefaultPlotParams(col=i,marker=22)
            pparams['Legend']='Tight Isolation'
            pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i]
            pparams['Options']='pe'
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = False ;aplot.scaleFactor =None
            aplot.doFit=False ; aplot.drawFit=True
            aplot.verbose= False
            plots[-1].addPlot(aplot)


        canvas = []
        for plot in plots:
            canvas.append(plot.plot())
